### OVERALL LOOK AT THE DATASET AND SILVER LAYER

In [1]:
import pandas as pd
import numpy as np
import pymongo
import re
from jproperties import Properties
from sqlalchemy import create_engine, text
from sqlalchemy import Numeric
import subprocess


# pandas settings
pd.set_option('max.colwidth', 200)
pd.set_option('display.max_rows', 50)

In [2]:
# loading variables from file (db access)

with open("app.properties", "r+b") as f:
    p = Properties()
    p.load(f, "utf-8")

### MongoDB Acess (Data Source)

In [3]:

client = pymongo.MongoClient()
db = client.get_database('zap_imoveis')
bronze_layer = db.get_collection('bronze')

In [4]:
print(f'Number of data records in the database: {bronze_layer.count_documents({})}')

Number of data records in the database: 20735


### Creating a DataFrame


In [5]:

df = pd.DataFrame(list(bronze_layer.find({})))

assert df[df['data-id'].duplicated()]['data-id'].sum() == 0, 'We have duplicated ids (data-id) in the DB'

In [6]:
df.head(2)

,_id,link,data-id,atts,date
0,67a344c52b111f021af8a118,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-2-quartos-com-piscina-saude-zona-sul-sao-paulo-sp-112m2-id-2774950239/,2774950239,"{'ad_type': 'Aluguel', 'sell_price': None, 'rent_price': 'R$ 5.000/mês', 'condo_price': 'R$ 970', 'address': 'Rua Doutor Samuel Porto, 237 - Saúde, São Paulo - SP', 'rstate_type': 'Apartamentos pa...",20250224
1,67a344cc2b111f021af8a119,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-1-quarto-com-piscina-cambuci-zona-sul-sao-paulo-sp-37m2-id-2733414161/,2733414161,"{'ad_type': 'Aluguel', 'sell_price': None, 'rent_price': 'R$ 2.400/mês', 'condo_price': 'R$ 980', 'address': 'Rua Backer, 338 - Cambuci, São Paulo - SP', 'rstate_type': 'Apartamentos para Alugar/'...",20250224


### Getting Only The Attributes  

In [7]:
df_atts =  pd.concat(
        [df[['data-id','link']], pd.json_normalize(df['atts'])],
        axis = 1
)

df_atts.head(2)

,data-id,link,ad_type,sell_price,rent_price,condo_price,address,rstate_type,floorSize,numberOfRooms,...,BEAUTY_ROOM,VINYL_FLOOR,DRESS_ROOM,CORRAL,DRYWALL,GOLF_FIELD,SMART_CONDOMINIUM,ECO_CONDOMINIUM,DIVIDERS,CARPET
0,2774950239,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-2-quartos-com-piscina-saude-zona-sul-sao-paulo-sp-112m2-id-2774950239/,Aluguel,None,R$ 5.000/mês,R$ 970,"Rua Doutor Samuel Porto, 237 - Saúde, São Paulo - SP",Apartamentos para Alugar/,112 m²,2 quartos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2733414161,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-1-quarto-com-piscina-cambuci-zona-sul-sao-paulo-sp-37m2-id-2733414161/,Aluguel,None,R$ 2.400/mês,R$ 980,"Rua Backer, 338 - Cambuci, São Paulo - SP",Apartamentos para Alugar/,37 m²,1 quarto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
## removing comercial as we are not interested for this analysis;

df_atts = df_atts[~df_atts.rstate_type.apply(lambda x: True if re.search('Comerciais', str(x)) else False)]
print(df_atts.rstate_type.unique())

['Apartamentos para Alugar/' 'Flats para Alugar/' 'Imóveis para Alugar/'
 'Casas para Alugar/' 'Apartamentos à Venda/' 'Coberturas à Venda/'
 'Sobrados à Venda/' 'Casas de Condomínio para Alugar/'
 'Sobrados para Alugar/' 'Studios para Alugar/'
 'Casas de Condomínio à Venda/' 'Coberturas para Alugar/' 'Casas à Venda/'
 'Casas de Vila à Venda/' 'Kitnets para Alugar/' 'Flats à Venda/'
 'Studios à Venda/' 'Casas de Vila para Alugar/' 'Lofts para Alugar/'
 'Imóveis à Venda/' 'Fazendas, Sítios e Chácaras para Alugar/'
 'Fazendas, Sítios e Chácaras à Venda/' 'Lofts à Venda/']


### Selecting Columns Of Interest

In [9]:
# for _ in df_atts.columns:
#      print(_)

cols_of_interest = [
    'data-id',
    'ad_type', 'rstate_type',
    'rent_price', 'condo_price', 'sell_price',
    'address', 'floorSize', 'numberOfBathroomsTotal',
    'numberOfParkingSpaces',
    'numberOfRooms',
    'numberOfSuites',
    'FURNISHED',
    'AIR_CONDITIONING',
    'HOME_OFFICE',
    'POOL',
    'GYM',
    'SAUNA',
    'LAUNDRY',
    'COWORKING',
    'link'
    ]

# keep only the cols that exist in the dataset; 

cols_of_interest = [x for x in cols_of_interest if x in df_atts.columns]

# change the df
df_atts = df_atts[cols_of_interest]


### % of NaN Values

In [10]:
missing = df_atts.isna().sum()
missing = missing/df_atts.shape[0] * 100
missing.sort_values(ascending=False)

## comments; 
# the null values on the ameneties represents the absence of such elements like GYM or LAUNDRY.
# for others that are mandatory (like at least sell or rent price and size), its more likely to be an error that occurred when web scraping. 

COWORKING                 99.276132
SAUNA                     90.975775
HOME_OFFICE               89.711418
GYM                       88.369848
LAUNDRY                   87.742496
FURNISHED                 83.640575
AIR_CONDITIONING          80.055014
POOL                      72.652254
sell_price                67.068816
numberOfSuites            19.327285
numberOfParkingSpaces      6.770582
numberOfRooms              0.357108
rent_price                 0.086864
numberOfBathroomsTotal     0.009652
rstate_type                0.000000
ad_type                    0.000000
data-id                    0.000000
condo_price                0.000000
floorSize                  0.000000
address                    0.000000
link                       0.000000
dtype: float64

In [11]:
df_atts[df_atts.numberOfRooms.isna()]

,data-id,ad_type,rstate_type,rent_price,condo_price,sell_price,address,floorSize,numberOfBathroomsTotal,numberOfParkingSpaces,...,numberOfSuites,FURNISHED,AIR_CONDITIONING,HOME_OFFICE,POOL,GYM,SAUNA,LAUNDRY,COWORKING,link
627,2732561794,Aluguel,Casas para Alugar/,R$ 7.100/mês,não informado,None,"Rua Amaro Guerra, 781 - Vila São Francisco Zona Sul, São Paulo - SP",180 m²,3 banheiros,8 vagas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-vila-sao-francisco-zona-sul-sao-paulo-180m2-id-2732561794/
655,2774886476,Aluguel,Casas para Alugar/,R$ 14.000/mês,isento,None,"Rua Euclides Pacheco, 350 - Tatuapé, São Paulo - SP",500 m²,7 banheiros,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-vila-gomes-cardim-sao-paulo-500m2-id-2774886476/
755,2763884729,Aluguel,Casas para Alugar/,R$ 12.500/mês,não informado,None,"Rua Doutor José Gustavo Bush, 350 - Panamby, São Paulo - SP",200 m²,6 banheiros,4 vagas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-paraiso-do-morumbi-sao-paulo-200m2-id-2763884729/
1738,2727319668,Aluguel,Casas para Alugar/,R$ 5.800/mês,não informado,None,"Rua Monte Serrat, 941 - Tatuapé, São Paulo - SP",120 m²,2 banheiros,1 vaga,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-com-area-de-servico-tatuape-sao-paulo-120m2-id-2727319668/
2332,2777283759,Venda,Casas à Venda/,R$ 24.000/mês,isento,R$ 4.500.000,"Alameda Joaquim Eugênio de Lima, 1489 - Jardim Paulista, São Paulo - SP",375 m²,4 banheiros,6 vagas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/venda-casa-jardim-paulista-sao-paulo-375m2-id-2777283759/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19204,2775826735,Aluguel,Casas para Alugar/,R$ 12.000/mês,R$ 1,None,"Avenida Santa Catarina - Vila Mascote, São Paulo - SP",300 m²,2 banheiros,2 vagas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-com-administracao-vila-mascote-sao-paulo-300m2-id-2775826735/
19660,2734867234,Aluguel,Casas para Alugar/,R$ 9.000/mês,R$ 1,None,"Rua Padre Carvalho - Pinheiros, São Paulo - SP",90 m²,2 banheiros,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-pinheiros-sao-paulo-90m2-id-2734867234/
19696,2748077358,Aluguel,Casas para Alugar/,R$ 7.500/mês,isento,None,"Rua Apucarana, 474 - Tatuapé, São Paulo - SP",176 m²,2 banheiros,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-com-cozinha-tatuape-sao-paulo-176m2-id-2748077358/
20377,2775827023,Aluguel,Casas para Alugar/,R$ 6.000/mês,R$ 1,None,"Rua Francisco Cruz - Vila Mariana, São Paulo - SP",169 m²,1 banheiro,1 vaga,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-com-administracao-vila-mariana-sao-paulo-169m2-id-2775827023/


### Removing Null Values
<li> if m2 = 0/NA or both rent and sell price are 0/NA, we'll remove the instance


In [ ]:
df_atts.shape

In [ ]:
df_atts = df_atts[~df_atts.floorSize.isnull()]

df_atts = df_atts[
    ~(df_atts.rent_price.isnull()) |
    ~(df_atts.sell_price.isnull())
    ]

In [ ]:
df_atts.shape

### Data Transformation 

<li> Dtypes


In [ ]:
# FLOAT 

def func(x):
    p = re.compile('[\d\.\,\- ]+')

    match = p.search(str(x))

    if match:
        return match.group(0).replace('.', '')
    else:
        return x



cols_float = ['sell_price', 'rent_price', 'condo_price']

df_atts = df_atts.replace({'isento': np.nan,'não informado': np.nan})

df_atts[cols_float] = df_atts[cols_float].map(func).replace('.', '')

df_atts[cols_float] = df_atts[cols_float].astype(float)
      

<li> Boolean


In [ ]:

cols_bool = ['AIR_CONDITIONING', 'FURNISHED','HOME_OFFICE', 'POOL','GYM', 'LAUNDRY', 'SAUNA', 'COWORKING'] # desired;

cols_bool = [col for col in cols_bool if col in df_atts.columns] # ajusted in relation to the dataset;



df_atts.loc[:, cols_bool] = df_atts[cols_bool].fillna(0) # 0 represents the unpresence of such property; 
df_atts.loc[:, cols_bool]= df_atts[cols_bool].replace(to_replace= ('^(?!0).*'), value = 1, regex= True) # all the rest can be replaced by 1 (True)

<li> Integer


In [ ]:

cols_int = ['floorSize', 'numberOfBathroomsTotal', 'numberOfParkingSpaces', 'numberOfRooms'] + cols_bool
df_atts[cols_int] = df_atts[cols_int].map(func)
df_atts[cols_int] = df_atts[cols_int].fillna(0).astype(int)

### Feature Engineering


In [ ]:
def regex_neighbor(x):

    match=re.search('(?<=- )[\w ]+', str(x))
    if match:
        return match.group()
    else:
        return x

df_atts.loc[:, 'neighborhood'] = df_atts.address.apply(regex_neighbor)
df_atts.loc[:, 'numberOfSuites'] = df_atts.numberOfSuites.fillna(0).apply(lambda x: re.search('\d+', str(x)).group())
df_atts['total_rental_price'] = df_atts.condo_price + df_atts.rent_price

# to int;
df_atts['numberOfSuites'] = df_atts.numberOfSuites.astype(int)

In [ ]:
for index, cols in df_atts.iterrows():
    
    if not isinstance(cols.address, float):
        address = cols.address.split(',')
        
        n = len(address)

        df_atts.loc[index, 'neighborhood']  = regex_neighbor(address[-2])

        if n == 3: # means we have street name;

            df_atts.loc[index, 'street'] = address[-3]
            df_atts.loc[index, 'number'] = address[-2].split('-')[0].strip()

  
df_atts = df_atts.drop(columns = ['address']) # we can drop; 

In [ ]:
df_atts.dtypes

### Data Cleaning

In [ ]:
df_atts.loc[:, 'rstate_type'] = df_atts.rstate_type.str[:-1]  # removing '/' at final of each string;


In [ ]:


cols_reord = [
             'data-id', 'rstate_type', 'ad_type',
             'rent_price',  'sell_price',  'condo_price', 'total_rental_price', 'floorSize',
             'neighborhood', 'street', 'number'] + cols_bool + \
            ['numberOfRooms', 'numberOfSuites', 'numberOfBathroomsTotal', 'numberOfParkingSpaces', 'link']

df_atts = df_atts[cols_reord]


In [ ]:
renamed_cols = {'data-id': 'id', 
                'rstate_type': 'kind', 'ad_type': 'rent_or_selling', 
                'rent_price':'price_rent', 'sell_price': 'price_sale', 'condo_price': 'price_condominium', 'total_rental_price': 'rent_plus_condo', 
                'floorSize': 'floor_size', 'number': 'address_number',  
                'AIR_CONDITIONING': 'air_conditioning', 'FURNISHED': 'furnished', 'HOME_OFFICE': 'home_office', 
                "POOL": 'pool', "GYM": 'gym', "LAUNDRY": 'laundry', 'SAUNA': 'sauna',
                'numberOfRooms': 'rooms', 'numberOfSuites': 'suites', 'numberOfBathroomsTotal': 'bathrooms', 'numberOfParkingSpaces': 'parking'}


df_atts = df_atts.rename(columns = renamed_cols)

In [ ]:
df_atts.head(2)

### Validating Duplicated Listing

In [ ]:
# Needed because the same house can be listed by different real estate agencies

columns = list(df_atts.columns)
for i in ['id', 'link']:
    columns.remove(i)

print(f'Number of duplicated listing based on REstate properties: {df_atts.duplicated(subset= columns).sum()}')

In [ ]:
df_atts = df_atts.drop_duplicates(subset=columns)

df_atts.shape

### Estatistical Cleaning

<li> outliers

In [ ]:
df_atts_sell = df_atts[~df_atts.price_sale.isna()]
df_atts_rent = df_atts[~df_atts.price_rent.isna()]  # as some adtype listed as "For Sale" may contain the price and be able to rent, we can use .isna()

In [ ]:
def plot_box_plot(df, sell_or_rent):

    if sell_or_rent == 'rent':
        box_data = df.price_rent / df.floor_size

    elif sell_or_rent == 'sell':
        box_data = df.price_sale / df.floor_size
    
    else:
        raise NameError

    Q1 = np.percentile(box_data, 25)
    Q3 = np.percentile(box_data, 75)
    IQR = Q3 - Q1

    u_bound = Q3 + 1.5*IQR
    l_bound = Q1 - 1.5*IQR

    return {'u_bound': u_bound, 'l_bound': l_bound}

In [ ]:
# IQR to remove outliers based on price/floor_size

rent_bounds = plot_box_plot(df_atts_rent, 'rent')
df_atts_rent_adjusted = df_atts_rent[
    ((df_atts_rent.price_rent / df_atts_rent.floor_size) < rent_bounds.get('u_bound')) &
   ( (df_atts_rent.price_rent / df_atts_rent.floor_size) > rent_bounds.get('l_bound') )
]

sale_bounds = plot_box_plot(df_atts_sell, 'sell')
df_atts_sale_adjusted = df_atts_sell[
    ((df_atts_sell.price_rent / df_atts_sell.floor_size) < sale_bounds.get('u_bound')) &
   ( (df_atts_sell.price_rent / df_atts_sell.floor_size) > sale_bounds.get('l_bound') )
]


In [ ]:
# making a new index to slice the original dataframe;
new_index = np.concatenate((df_atts_rent_adjusted.index,df_atts_sale_adjusted.index))
new_index = list(set(new_index))

df_atts = df_atts.loc[new_index, :]

In [ ]:
df_atts

In [ ]:
hld

### Postgres Ingestion;


In [ ]:
# db credentials come from the file app.properties

uri = p.get('uri').data
engine = create_engine(uri)

with engine.connect() as con:
    con.execute(text("CREATE SCHEMA if not exists zap"))
    con.commit()

In [ ]:
# drop views to update silver table; 
with engine.connect() as conn:
    views_list = conn.execute(
    text("""select table_name 
            from information_schema.views
            where table_schema = 'zap' """)).fetchall() # getting the view's name that exists;


    for view in views_list:  # dropping them because it's needed;
        conn.execute(text(f"DROP VIEW zap.{view[0]}"))
        conn.commit() 


df_atts.to_sql('silver_zapimoveis', schema='zap', # now its gonna work;
    con = uri, if_exists='replace', index = False, 
    dtype={'price_rent': Numeric, 'price_sale': Numeric, 'price_condominium': Numeric, 'rent_plus_condo': Numeric})


In [ ]:
### call run dbt;
subprocess.run("dbt run", shell=True)

### END